# GitHub RAG - Vector Manager


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

LangChain 추적


In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG - Vector Manager"

In [3]:
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

### 데이터 준비


GitHub Repository 를 다운로드 받습니다. 이번 예제는 langchain-ai 공식 Repository 로 진행합니다.

- !git clone 을 사용하여 Repository clone

다른 저장소의 파일을 사용하고 싶다면, `root_dir`을 여러분의 저장소의 루트 디렉토리로 변경하세요.


저는 `/Users/teddy/Dev/github/langchain` 위치에 `langchain` reposotory 를 clone 하였습니다. 아래의 경로는 본인의 경로에 맞게 바꾸어 주어야 합니다.


In [4]:
#!ls "/Users/teddy/Dev/github/langchain/libs"
!ls "/home/charles/ai_project/langchain/libs"


cli	   core		 langchain  standard-tests
community  experimental  partners   text-splitters


## 도큐먼트 로더


여기서 모든 패키지의 파일을 불러오지 않습니다. 핵심 기능을 포함하는 특정 폴더의 파일만 불러오도록 아래와 같이 정의해 주었습니다.


In [5]:
# Root 경로
# repo_root = "/Users/teddy/Dev/github/langchain/libs"
# repo_root = "/Users/julio/projects/gpt-projects/langchain/libs"
repo_root = "/home/charles/ai_project/langchain/libs"

# 불러오고자 하는 패키지 경로
repo_core = repo_root + "/core/langchain_core"
repo_community = repo_root + "/community/langchain_community"
repo_experimental = repo_root + "/experimental/langchain_experimental"
repo_parters = repo_root + "/partners"
repo_text_splitter = repo_root + "/text_splitters/langchain_text_splitters"
# repo_cookbook = repo_root + "/cookbook"

In [6]:
# langchain의 여러 모듈을 가져옵니다.
from langchain_text_splitters import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

# 불러온 문서를 저장할 빈 리스트를 생성합니다.
py_documents = []

for path in [repo_core, repo_community, repo_experimental, repo_parters]:
    # GenericLoader를 사용하여 파일 시스템에서 문서를 로드합니다.
    loader = GenericLoader.from_filesystem(
        path,  # 문서를 불러올 경로
        glob="**/*",  # 모든 하위 폴더와 파일을 대상으로 함
        suffixes=[".py"],  # .py 확장자를 가진 파일만 대상으로 함
        parser=LanguageParser(
            language=Language.PYTHON, parser_threshold=30
        ),  # 파이썬 언어의 문서를 파싱하기 위한 설정
    )
    # 로더를 통해 불러온 문서들을 documents 리스트에 추가합니다.
    py_documents.extend(loader.load())

print(f".py 파일의 개수: {len(py_documents)}")

<unknown>:54: SyntaxWarning: invalid escape sequence '\s'
<unknown>:54: SyntaxWarning: invalid escape sequence '\s'
<unknown>:54: SyntaxWarning: invalid escape sequence '\s'
<unknown>:321: SyntaxWarning: invalid escape sequence '\{'
<unknown>:321: SyntaxWarning: invalid escape sequence '\{'
<unknown>:321: SyntaxWarning: invalid escape sequence '\{'
<unknown>:147: SyntaxWarning: invalid escape sequence '\M'
<unknown>:147: SyntaxWarning: invalid escape sequence '\M'
<unknown>:147: SyntaxWarning: invalid escape sequence '\M'
<unknown>:31: SyntaxWarning: invalid escape sequence '\s'
<unknown>:31: SyntaxWarning: invalid escape sequence '\s'
<unknown>:31: SyntaxWarning: invalid escape sequence '\s'


.py 파일의 개수: 5830


다음은 `.mdx` 확장자를 가진 파일을 `TextLoader` 를 사용하여 불러옵니다. `.mdx` 파일은 Jupyter Notebook 파일을 마크다운 형식으로 변환한 파일이며, 유용한 예제를 포함하고 있으므로 이를 DB 에 추가하기 위해 도큐먼트 형식으로 로드압니다.


In [7]:
import os

# TextLoader 모듈을 불러옵니다.
from langchain_community.document_loaders import TextLoader

# 검색할 최상위 디렉토리 경로를 정의합니다.
# root_dir = "/Users/teddy/Dev/github/langchain/"
root_dir = "/home/charles/ai_project/langchain/"


mdx_documents = []
# os.walk를 사용하여 root_dir부터 시작하는 모든 디렉토리를 순회합니다.
for dirpath, dirnames, filenames in os.walk(root_dir):
    # 각 디렉토리에서 파일 목록을 확인합니다.
    for file in filenames:
        # 파일 확장자가 .mdx인지 확인하고, 경로 내 '*venv/' 문자열이 포함되지 않는지도 체크합니다.
        if (file.endswith(".mdx")) and "*venv/" not in dirpath:
            try:
                # TextLoader를 사용하여 파일의 전체 경로를 지정하고 문서를 로드합니다.
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                # 로드한 문서를 분할하여 documents 리스트에 추가합니다.
                mdx_documents.extend(loader.load())
            except Exception:
                # 파일 로드 중 오류가 발생하면 이를 무시하고 계속 진행합니다.
                pass

# 최종적으로 불러온 문서의 개수를 출력합니다.
print(f".mdx 파일의 개수: {len(mdx_documents)}")

.mdx 파일의 개수: 272


## Chunk 분할


파일들을 청크로 나누어 봅시다.


In [8]:
# RecursiveCharacterTextSplitter 모듈을 가져옵니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter


# RecursiveCharacterTextSplitter 객체를 생성합니다. 이 때, 파이썬 코드를 대상으로 하며,
# 청크 크기는 2000, 청크간 겹치는 부분은 200 문자로 설정합니다.
py_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=1000, chunk_overlap=100
)

# py_docs 변수에 저장된 문서들을 위에서 설정한 청크 크기와 겹치는 부분을 고려하여 분할합니다.
py_docs = py_splitter.split_documents(py_documents)

# 분할된 텍스트의 개수를 출력합니다.
print(f"분할된 .py 파일의 개수: {len(py_docs)}")





mdx_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)

# mdx_docs 변수에 저장된 문서들을 위에서 설정한 청크 크기와 겹치는 부분을 고려하여 분할합니다.
mdx_docs = mdx_splitter.split_documents(mdx_documents)

# 분할된 텍스트의 개수를 출력합니다.
print(f"분할된 .mdx 파일의 개수: {len(mdx_docs)}")

분할된 .py 파일의 개수: 17103
분할된 .mdx 파일의 개수: 1092


로드한 도큐먼트를 합칩니다.


In [9]:
combined_documents = py_docs + mdx_docs
print(f"총 도큐먼트 개수: {len(combined_documents)}")


총 도큐먼트 개수: 18195


## [추가됨] Combind Document to Local File(Store)

In [10]:
import os
import pickle

# 문서들을 로컬 파일에 저장하는 함수
def save_documents_to_file(documents, file_path):
    # 파일 경로의 디렉토리를 추출
    directory = os.path.dirname(file_path)
    
    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(file_path, 'wb') as f:
        pickle.dump(documents, f)
    print(f"Documents saved to {file_path}")

# 문서들을 로컬 파일에 저장
save_documents_to_file(combined_documents, './documents/combined_documents.pkl')


Documents saved to ./documents/combined_documents.pkl


## Embedding


In [11]:
# langchain_openai와 langchain의 필요한 모듈들을 가져옵니다.
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 로컬 파일 저장소를 사용하기 위해 LocalFileStore 인스턴스를 생성합니다.
# './cache/' 디렉토리에 데이터를 저장합니다.
store = LocalFileStore("./cache/")

# OpenAI 임베딩 모델 인스턴스를 생성합니다. 모델명으로 "text-embedding-3-small"을 사용합니다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", disallowed_special=())

# CacheBackedEmbeddings를 사용하여 임베딩 계산 결과를 캐시합니다.
# 이렇게 하면 임베딩을 여러 번 계산할 필요 없이 한 번 계산된 값을 재사용할 수 있습니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model
)

## Vector DB


- `from_documents`: 이 메서드는 문서 컬렉션과 이에 해당하는 임베딩을 받아 벡터 인덱스를 생성합니다. 여기서는 `combined_documents` 로부터 벡터를 생성하고, 이 벡터들은 `cached_embeddings` 을 통해 임베딩된 데이터를 사용합니다.
- `save_local`: 이 메서드는 생성된 FAISS 인덱스를 지정된 로컬 폴더에 저장합니다. 이 폴더명은 `FAISS_DB_INDEX` 변수에 저장되어 있습니다.


In [12]:
# langchain_community 모듈에서 FAISS 클래스를 가져옵니다.
from langchain_community.vectorstores import FAISS

# 로컬에 저장할 FAISS 인덱스의 폴더 이름을 지정합니다.
FAISS_DB_INDEX = "langchain_faiss"

# combined_documents 문서들과 cached_embeddings 임베딩을 사용하여
# FAISS 데이터베이스 인스턴스를 생성합니다.
db = FAISS.from_documents(combined_documents, cached_embeddings)

# 생성된 데이터베이스 인스턴스를 지정한 폴더에 로컬로 저장합니다.
db.save_local(folder_path=FAISS_DB_INDEX)

FAISS 데이터베이스 및 Document의 저장이 정상적으로 완료되었습니다.
